In [13]:
from __future__ import print_function, division

# Standard imports to work with datasets, plots, etc.
import pylab as pl
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import json
import requests
import urllib2
from getCitiBikeCSV import getCitiBikeCSV
from get_jsonparsed_data import get_jsonparsed_data
from pandas.tools.plotting import scatter_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy.interpolate import *

#loading rcParams from JSON template (created from Federica's fbb_matplotlibrc.json)
url = 'https://s3.amazonaws.com/sb-public/sbg389_matplotlibrc.json'
resp = requests.get(url=url)
data = json.loads(resp.text)
# update the rcParams object
pl.rcParams.update(data)

#loading API Keys
jsondata = open("keys.json").read()
keys = json.loads(jsondata)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [17]:
key=(keys['googlekey'])
#print (key)

In [14]:
datestring = '201501'
#getCitiBikeCSV(datestring)
df_W = pd.read_csv(os.getenv('PUIDATA')+'/' + datestring + '-citibike-tripdata.csv')
print (df_W.columns)
datestring = '201506'
#getCitiBikeCSV(datestring)
df_S = pd.read_csv(os.getenv ('PUIDATA')+'/' + datestring + '-citibike-tripdata.csv')
print (df_W.columns)


Index([u'tripduration', u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year', u'gender'],
      dtype='object')
Index([u'tripduration', u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year', u'gender'],
      dtype='object')


In [12]:
year = int(datestring[:4])

#but really you want the age of rider when they were riding:
year = int(datestring[:4])
print (year)

#df is the dataframe where the content of the csv file is stored
df_W['age'] = year - df_W['birth year'][(df_W['usertype'] == 'Subscriber')]
df_S['age'] = year - df_S['birth year'][(df_S['usertype'] == 'Subscriber')]

agelonglat = pd.concat([df_W, df_S])

2015


In [5]:
agelonglat = agelonglat[['age','start station longitude','start station latitude']]
agelonglat.age.dropna(inplace=True)

In [53]:
agelonglat.head()

a = (agelonglat[['start station latitude', 
                'start station longitude']].values.astype(float))

len(a)
zipcodesDF = pd.DataFrame()

# Fill it in with zeros
agelonglat['zipcodes'] = np.zeros(len(agelonglat))

for latlon in pd.DataFrame(a).drop_duplicates().values:
    url = ("https://maps.googleapis.com/maps/api/geocode/json?latlng=" +
           "%f,%f&key=%s"%(
            latlon[0], latlon[1], key))
    
    #print (latlon, url)
    #print (get_jsonparsed_data(url))
    revgeo = get_jsonparsed_data(url)["results"][0]['address_components'][-1]
    
    #print (revgeo)
    agelonglat['zipcodes'][(agelonglat['start station latitude'] == latlon[0]) * 
           (agelonglat['start station longitude'] == latlon[1])] = revgeo['long_name']

agelonglat.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,age,start station longitude,start station latitude,zipcodes
0,55.0,-73.969053,40.750020,10017
1,52.0,-74.003664,40.743174,10011
2,41.0,-73.986022,40.740964,10010
3,46.0,-73.965964,40.683178,2702
4,38.0,-73.986831,40.745168,10016


In [60]:
agelonglat.head()

#len(agelonglat)

,age,start station longitude,start station latitude,zipcodes
0,55.0,-73.969053,40.750020,10017
1,52.0,-74.003664,40.743174,10011
2,41.0,-73.986022,40.740964,10010
3,46.0,-73.965964,40.683178,2702
4,38.0,-73.986831,40.745168,10016


In [63]:
#pd.DataFrame(a).drop_duplicates()

In [67]:
ksAges = stats.kstest(agelonglat[::500], 'norm', args=(agelonglat.age.mean(), agelonglat.age.std()))

TypeError: unsupported operand type(s) for -: 'unicode' and 'float'